# **Algorithm**

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Fundamental classes
import numpy as np
import pandas as pd
import tensorflow as tf
import os

# Image related
import cv2
from PIL import Image

#Performance Plot
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For the model and it's training
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, Add
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Time
import time
import datetime

# Ultralytics
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image
import torch
from ultralytics import YOLO

# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def inicialitzacio():
  # Room Classification Model
  room_classification_model = YOLO('/content/drive/MyDrive/TFG/yolo-room-classification/training/train/weights/last.pt')

  # Obj segmentation models

  rooms = ['bathroom', 'kitchen', 'inside']

  dicc_obj_seg_models = {}

  for room in rooms:

    dicc_obj_seg_models[room] = YOLO(f'/content/drive/MyDrive/TFG/{room}-obj-seg/training-results-yolo/train01/weights/last.pt')

  # Obj classification models

  rooms = ['bathroom', 'kitchen', 'inside']
  classes = {'kitchen':['sink', 'floor', 'cabinet', 'counter', 'tiles'],
             'bathroom':['sink', 'floor', 'cabinet', 'shower', 'tiles'],
             'inside':['floor', 'paint', 'window']}
  models = ['resnet', 'simple_cnn', 'vgg']

  dicc_obj_class_models = {}

  for room in rooms:

    models_class = {}

    for mod in models:
      dic = {}

      for classe in classes[room]:
        dic[classe] = load_model(f'/content/drive/MyDrive/TFG/{room}-obj-classification/models/{mod}_model_{classe}.h5')

      models_class[mod] = dic

    dicc_obj_class_models[room] = models_class

  return room_classification_model, dicc_obj_seg_models, dicc_obj_class_models




In [ ]:
def room_classification(model, img_path):

  results = model(img_path)

  probs = results[0].probs.data.cpu().numpy() # convertim les probabilitats de cada classe en un array de numpy
  predicted_class_idx = probs.argmax()  # Índex de la classe amb la probabilitat més alta
  predicted_class = results[0].names[predicted_class_idx]  # Nom de la classe
  predicted_confidence = probs[predicted_class_idx]  # Confiança de la predicció

  print(f'\nClasse predita: {predicted_class} amb una confiança de {predicted_confidence:.2f}')

  return predicted_class

In [ ]:

def obj_segmentation(dicc_models, room, img_path):
  # Run inference on an image
  results_seg = dicc_models[room](img_path)  # results list

  # Image size
  img = cv2.imread(img_path)
  H, W, _ = img.shape

  # Define colors for each class
  class_colors = {}

  # Generate a unique color for each class
  def get_color_for_class(class_id):
      if class_id not in class_colors:
          class_colors[class_id] = np.random.uniform(0, 255, size=3)
      return class_colors[class_id]

  # Initialize list to store bounding box information
  bounding_boxes_info = []

  # View results
  for result in results_seg:
      for j, (mask, box, cls, conf) in enumerate(zip(result.masks.data, result.boxes.xyxy, result.boxes.cls, result.boxes.conf)):
          # Convert mask to numpy array and scale to [0, 255]
          mask = mask.cpu().numpy() * 255

          # Optionally resize mask to original image size
          # mask = cv2.resize(mask, (W, H))

          # Get bounding box coordinates
          x_min, y_min, x_max, y_max = box.cpu().numpy().astype(int)

          # Get class name and confidence
          class_name = result.names[int(cls)]
          confidence = conf.cpu().numpy()

          # Only draw the bounding box and label if confidence is greater than 0.5
          if confidence > 0.5:
              # Get color for the current class
              color = get_color_for_class(int(cls))

              # Draw bounding box on the image
              cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 3)

              # Put class name and confidence text above the bounding box
              label = f"{class_name}: {confidence:.2f}"
              label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.7, 2)
              top = max(y_min, label_size[1])
              cv2.rectangle(img, (x_min, top - label_size[1]), (x_min + label_size[0], top + 5), color, cv2.FILLED)
              cv2.putText(img, label, (x_min, top), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 0), 2)

              # Append bounding box info to the list
              bounding_boxes_info.append([img_path, class_name, x_min, y_min, x_max, y_max])

  # Save the image with bounding boxes and labels
  # cv2.imwrite('/content/drive/MyDrive/TFG/algorithm/outputs/image_with_boxes_and_labels.png', img)

  # Convert bounding box info to a DataFrame
  bounding_boxes_df = pd.DataFrame(bounding_boxes_info, columns=['filename', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])

  # Save the DataFrame to a CSV file
  # bounding_boxes_df.to_csv(csv_path, index=False)
  # print(bounding_boxes_df)

  # Convert BGR image to RGB for display
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # # Display the image
  # plt.imshow(img_rgb)
  # plt.axis('off')  # Hide axis
  # plt.show()

  return bounding_boxes_df, img_rgb

In [ ]:
# obj classification

# Prepare data for prediction
def prepare_data_for_prediction(annotations_df, target_size=(150, 150)):
    images = []
    regions_info = []
    for _, row in annotations_df.iterrows():
        img_path = row['filename']
        image = load_img(img_path, color_mode='grayscale')
        image = img_to_array(image)
        x_min = int(row['xmin'])
        y_min = int(row['ymin'])
        x_max = int(row['xmax'])
        y_max = int(row['ymax'])
        classe = row['class']
        region = image[y_min:y_max, x_min:x_max]
        region = cv2.resize(region, target_size)
        region = region / 255.0  # Normalize
        images.append(region)
        regions_info.append((img_path, x_min, y_min, x_max, y_max, classe))
    return np.array(images), regions_info

def obj_classification( dicc, room, image_path, annotations_df):

  # Prepare the data for prediction
  images, regions_info = prepare_data_for_prediction(annotations_df)

  models = ['simple_cnn', 'vgg', 'resnet']

  # Make predictions

  predictions = []

  for img, info in zip(images, regions_info):
    votacions = 0
    img_batch = np.expand_dims(img, axis=0)  # Add batch dimension

    for mod in models:
      prediction = dicc[room][mod][info[-1]].predict(img_batch)
      prediction_class = 1 if prediction >= 0.5 else 0  # Assuming binary classification with sigmoid output
      # print(f"El model {mod} ha predit la classe {info[-1]} com a {'bona' if prediction_class == 1 else 'dolenta'}")

      votacions += prediction_class

    pred = f"La classe {info[-1]} s'ha predit com a {'bona' if votacions > 1 else 'dolenta'}"
    predictions.append((info[-1], 1 if votacions > 1 else 0))

  return predictions


In [ ]:
from PIL import Image
from IPython.display import display

def analize_img(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, IMG_PATH):

  img = Image.open(IMG_PATH)

  # Mostra la imatge
  display(img)


  predicted_room = room_classification(room_classification_model, IMG_PATH)

  if predicted_room == 'outside':
    return predicted_room, 0

  bounding_boxes_df, img_rgb = obj_segmentation(dicc_obj_seg_models, predicted_room, IMG_PATH)

  predictions = obj_classification(dicc_obj_class_models, predicted_room, IMG_PATH, bounding_boxes_df)


  classes_used = []
  print('--------------------\n')
  print("Resumint:\n")
  print(f'Tenim una imatge de la zona {predicted_room} on li hem trobat les següents classes:\n')
  # Display the image
  plt.imshow(img_rgb)
  plt.axis('off')  # Hide axis
  plt.show()

  for classe, pred in predictions:
    if classe in classes_used: continue
    print(f"La classe {classe} s'ha predit com a {'bona' if pred > 1 else 'dolenta'}")

  return predicted_room, predictions

In [ ]:
room_classification_model, dicc_obj_seg_models, dicc_obj_class_models = inicialitzacio()

# Provem 3 imatges

In [ ]:
IMG_PATH = '/content/drive/MyDrive/TFG/algorithm/prova_001.png'

analize_img(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, IMG_PATH)

In [ ]:
IMG_PATH = '/content/drive/MyDrive/TFG/algorithm/prova_002.png'

analize_img(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, IMG_PATH)

In [ ]:
IMG_PATH = '/content/drive/MyDrive/TFG/algorithm/prova_003.png'

analize_img(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, IMG_PATH)

# Algorisme A la Propietat

Ho implementem per una propietat completa

In [ ]:
l = [0]*2
l

In [ ]:
import math
import os


def analize_prpty(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, folder_path, nBath):

  dicc = {'bathroom': {'sink': [0]*2, 'tiles': [0]*2, 'floor': [0]*2, 'cabinet': [0]*2, 'shower': [0]*2}, 'kitchen': {'sink': [0]*2, 'tiles': [0]*2, 'floor': [0]*2, 'cabinet': [0]*2, 'counter': [0]*2}, 'inside': {'paint': [0]*2, 'window': [0]*2, 'floor': [0]*2}, 'outside': 0}

  # Itera sobre tots els fitxers de la carpeta
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):

        room_prediction, predictions = analize_img(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, file_path)

        print(predictions)

        if room_prediction == 'outside':
          dicc[room_prediction] += 1
        else:
          dicc[room_prediction][predictions[0][0]][predictions[0][1]] += 1

  reformes = {'bathroom': {}, 'kitchen': {} , 'inside': {}}

  # Bathrooms
  for amenity, pred in dicc['bathroom'].items():
    prob_amenity_dolenta = pred[0]/(pred[0] + pred[1])

    banys_amb_amenity_dolenta = round(prob_amenity_dolenta * nBath)

    if banys_amb_amenity_dolenta != 0:
      reformes['bathroom'][amenity] = banys_amb_amenity_dolenta


  # Kitche
  for amenity, pred in dicc['kitchen'].items():
    if (pred[0] + pred[1]) != 0:
      prob_amenity_dolenta = round(pred[0]/(pred[0] + pred[1]))

      if prob_amenity_dolenta != 0:
        reformes['kitchen'][amenity] = prob_amenity_dolenta


  # Inside
  for amenity, pred in dicc['inside'].items():
    if (pred[0] + pred[1]) != 0:
      prob_amenity_dolenta = pred[0]/(pred[0] + pred[1])

      if prob_amenity_dolenta != 0:
        reformes['inside'][amenity] = prob_amenity_dolenta

  print('----------\n')
  print('La propietat necessita: \n')

  # Bathroom
  for amenity, prob in reformes['bathroom'].items():
    print(f"Renovar {prob} de {nBath} {amenity} en el bany")

  # Kitchen
  for amenity, prob in reformes['kitchen'].items():
    print(f"Renovar {amenity} a la cuina")

  # Kitchen
  for amenity, prob in reformes['inside'].items():
    print(f"Renovar {prob}% de {amenity} de la propietat")

  return reformes




In [ ]:
# Defineix el camí de la carpeta
folder_path = '/content/drive/MyDrive/TFG/algorithm/propietat_prova/'

reformes = analize_prpty(room_classification_model, dicc_obj_seg_models, dicc_obj_class_models, folder_path, 2)

In [ ]:
round(5.3)